# pySand 1.5
##### Calculating erosion rate and transport velocity based on basic measurements - created with version 1.6


In [1]:
import pysand as ps
import numpy as np

## Input

Typical sensor input:

In [2]:
P = 40  # Pressure [bar]
T = 50  # Temperature [deg C]
Qo = 300  # Oil rate [Sm3/d]
Qw = 2300  # Water rate [Sm3/d]
Qg = 320000  # Gas rate [Sm3/d]
raw = 6450  # Acoustic response from ASD
zero = 3200  # Background noise (response without sand)

Fluid and particle input:

In [3]:
rho_o = 780  # Oil density at std conditions [kg/m3]
rho_w = 980  # Water density at std conditions [kg/m3]
mu_o = 2.5e-3  # Oil viscosity [kg/ms]
mu_w = 1e-3  # Water viscosity [kg/ms]
mu_g = 1e-5  # Gas viscosity [kg/ms]
MW = 19.2  # Gas molecular weight [kg/kmol]
Z = .95  # Gas compressibility factor [-]
d_p = .4  # Particle diameter [mm]

Geometry input:

In [4]:
D = 0.12 # Pipe cross sectional diameter [m]
GF = 3  # Geometry factor

## Calculations

Calculate the gas liquid ratio

In [5]:
GLR = Qg/(Qo+Qw)  # Gas liquid ratio [Sm3/Sm3]
print('GLR:', np.round(GLR,1), 'Sm3/Sm3')

GLR: 123.1 Sm3/Sm3


Calculate basic fluid properties

In [6]:
v_m = ps.fluidproperties.mix_velocity(P, T, Qo, Qw, Qg, Z, D)
rho_m = ps.fluidproperties.mix_density(P, T, Qo, Qw, Qg, rho_o, rho_w, MW, Z)
mu_m = ps.fluidproperties.mix_viscosity(P, T, Qo, Qw, Qg, mu_o, mu_w, mu_g, Z)
print('Mix velocity:', v_m, 'm/s')
print('Mix density:', rho_m, 'kg/m3')
print('Mix viscosity:', mu_m, 'kg/ms')

Mix velocity: 11.47 m/s
Mix density: 245.04 kg/m3
Mix viscosity: 0.00028 kg/ms


Calculate step value based on standard vendor model

In [7]:
step = ps.asd.std_step_clampon(v_m, GLR)
print('Step:', step)

Step: 7755.5


Calculate sand rate:

In [8]:
Q_s = ps.asd.sand_rate(raw, zero, step)
print('Sand rate:',np.round(Q_s,2),'g/s')

Sand rate: 0.42 g/s


Calculate the relative and aboslute erosion rates on an aluminium blinded tee:

In [9]:
E = ps.erosion.tee(v_m, rho_m, mu_m, GF, D, d_p, material='aluminium')
E_rate = ps.erosion.erosion_rate(E, Q_s)

Check that velocity is above critical transport velocity:

In [10]:
rho_l = (rho_w * Qw + rho_o * Qo) / (Qw + Qo)
mu_l = (mu_w * Qw + mu_o * Qo) / (Qo + Qw)
crit_v = ps.transport.hydro(D, rho_l, mu_l, d_p)

## Result

In [11]:
print('Relative erosion rate:', np.round(E,2), 'mm/ton')
print('Erosion rate:', np.round(E_rate, 2), 'mm/year')

Relative erosion rate: 0.04 mm/ton
Erosion rate: 0.48 mm/year


In [12]:
if v_m > crit_v[1]:
    print('The velocity is above the critical velocity for dispersed flow transport')
elif v_m > crit_v[0]:
    print('The velocity is between critical velocities for moving beds and dispersed flow')
else:
    print('The velocity is below critical velocity for transport')

The velocity is above the critical velocity for dispersed flow transport
